# BUY AND SELL THE NEXT DAY

- Buy the stock on the fourth day open, if the stock closes down consecutively for three days.
- Exit on the next day open.
- Optional: Optimize the strategy by exiting the long position on the same day close.

Import the required libraries

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [2]:
    #Create a class to backtest the strategy
    class backtest_buy_sell:
    
    #Create object attributes/methods
        def __init__(self,ticker,start,end):
                
            self.ticker = ticker
            self.start = start
            self.end = end
       
        
            self.fetch_data()
            self.indicators()
            self.signals()
            self.positions()
            self.returns()
        
    # Fetch the data from yahoo finance
        def fetch_data(self):
            self.df = yf.download(self.ticker,self.start,self.end)        
        
    # Compute the required indicators such as open - open returns,close - close returns and open - close returns
        def indicators(self):
            self.df['oo_returns'] = np.log(self.df['Open']/self.df['Open'].shift(1))
            self.df['cc_returns'] = np.log(self.df['Adj Close']/self.df['Adj Close'].shift(1))
            self.df['oc_returns'] = np.log(self.df['Adj Close']/self.df['Open'])
                                       
    # Compute the long only signals
        def signals(self):
            self.df['signals'] = np.where((self.df['cc_returns'].shift(1) < 0) & (self.df['cc_returns'].shift(2) < 0) & (self.df['cc_returns'].shift(3) < 0),1,0)              
              
    # Take the positions
        def positions(self):
            self.df['positions'] = self.df['signals'].shift(1)
        
                                                                         
        # Calculate the strategy returns
        def returns(self):
        
        #Calculate the strategy returns by considering buy at open price and close at next day open price
            self.df['Strategy_Returns'] = self.df['positions']*self.df['oo_returns']
            print('Strategy Returns :', np.round(self.df['Strategy_Returns'].cumsum()[-1], 2))
            print('Buy and Hold Returns :', np.round(self.df['cc_returns'].cumsum()[-1], 2))
            Strat_Returns = np.round(self.df['Strategy_Returns'].cumsum()[-1], 2)
            Buy_Hold_Returns = np.round(self.df['cc_returns'].cumsum()[-1], 2)
            return Strat_Returns, Buy_Hold_Returns

In [3]:
# Define the backtesting time period

end = pd.datetime.now().date()-pd.Timedelta(days=2*252)
start = end - pd.Timedelta(days=10*252)


In [4]:
# Generate the list of nifty50_stocks you want to backtest the strategy 
nifty50_stocks_list = ['BHARTIARTL.NS','NTPC.NS','MARUTI.NS','HINDALCO.NS','ONGC.NS','KOTAKBANK.NS','BAJAJ-AUTO.NS','COALINDIA.NS',
                      'ITC.NS','ULTRACEMCO.NS','CIPLA.NS','LT.NS','INDUSINDBK.NS','HDFCLIFE.NS','TATACONSUM.NS','RELIANCE.NS',
                      'TATASTEEL.NS','ICICIBANK.NS','GRASIM.NS','NESTLEIND.NS','TCS.NS','TECHM.NS','SHREECEM.NS','BAJFINANCE.NS',
                      'BAJAJFINSV.NS','WIPRO.NS','BRITANNIA.NS','HEROMOTOCO.NS','TITAN.NS'] 

nifty_stocks_name=[]
nifty_stocks_strategy_returns =[]

for i in nifty50_stocks_list:
    a = backtest_buy_sell(i,start,end)
    nifty_stocks_name.append(i)
    nifty_stocks_strategy_returns.append(a.returns())

Strat = [x[0] for x in nifty_stocks_strategy_returns]
BnH = [x[1] for x in nifty_stocks_strategy_returns]

[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.3
Buy and Hold Returns : 0.46
Strategy Returns : 0.3
Buy and Hold Returns : 0.46
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.02
Buy and Hold Returns : -0.08
Strategy Returns : -0.02
Buy and Hold Returns : -0.08
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.1
Buy and Hold Returns : 1.62
Strategy Returns : -0.1
Buy and Hold Returns : 1.62
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.62
Buy and Hold Returns : 0.46
Strategy Returns : 0.62
Buy and Hold Returns : 0.46
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.27
Buy and Hold Returns : -0.67
Strategy Returns : 0.27
Buy and Hold Returns : -0.67
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.45
Buy and Hold Returns : 1.32
Strategy R

In [5]:
# Convert the results into dataframe sorted in a order which delivers greater returns
Results = pd.DataFrame({'Nifty_50_Stocks':nifty_stocks_name,'Strategy Returns':Strat,'Buy and Hold Returns' : BnH})
Results.sort_values(by = 'Strategy Returns',ascending=False,inplace=True)

In [6]:
Results

,Nifty_50_Stocks,Strategy Returns,Buy and Hold Returns
12,INDUSINDBK.NS,0.67,0.39
21,TECHM.NS,0.67,0.81
3,HINDALCO.NS,0.62,0.46
23,BAJFINANCE.NS,0.60,3.34
24,BAJAJFINSV.NS,0.55,2.26
5,KOTAKBANK.NS,0.45,1.32
17,ICICIBANK.NS,0.45,0.80
20,TCS.NS,0.37,0.98
28,TITAN.NS,0.34,1.62
10,CIPLA.NS,0.33,0.58


Optimize the strategy by exiting the long position on the same day close

In [7]:
class backtest_buy_sell_same_day_close(backtest_buy_sell):

    def positions(self):
        self.df['positions'] = self.df['signals']
        
    def returns(self):
        self.df['Strategy_Returns'] = self.df['positions']*self.df['oc_returns']
        print('Strategy Returns :', np.round(self.df['Strategy_Returns'].cumsum()[-1], 2))
        print('Buy and Hold Returns :', np.round(self.df['cc_returns'].cumsum()[-1], 2))
        Strat_Returns = np.round(self.df['Strategy_Returns'].cumsum()[-1], 2)
        Buy_Hold_Returns = np.round(self.df['cc_returns'].cumsum()[-1], 2)
        return Strat_Returns

In [8]:
# Generate the list of nifty50_stocks you want to backtest the strategy 
nifty50_stocks_list = ['BHARTIARTL.NS','NTPC.NS','MARUTI.NS','HINDALCO.NS','ONGC.NS','KOTAKBANK.NS','BAJAJ-AUTO.NS','COALINDIA.NS',
                      'ITC.NS','ULTRACEMCO.NS','CIPLA.NS','LT.NS','INDUSINDBK.NS','HDFCLIFE.NS','TATACONSUM.NS','RELIANCE.NS',
                      'TATASTEEL.NS','ICICIBANK.NS','GRASIM.NS','NESTLEIND.NS','TCS.NS','TECHM.NS','SHREECEM.NS','BAJFINANCE.NS',
                      'BAJAJFINSV.NS','WIPRO.NS','BRITANNIA.NS','HEROMOTOCO.NS','TITAN.NS'] 

nifty_stocks_name_intraday=[]
nifty_stocks_strategy_returns_intraday =[]

for j in nifty50_stocks_list:
    b = backtest_buy_sell_same_day_close(j,start,end)
    nifty_stocks_name_intraday.append(j)
    nifty_stocks_strategy_returns_intraday.append(b.returns())
    

[*********************100%***********************]  1 of 1 completed
Strategy Returns : -4.38
Buy and Hold Returns : 0.46
Strategy Returns : -4.38
Buy and Hold Returns : 0.46
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -42.97
Buy and Hold Returns : -0.08
Strategy Returns : -42.97
Buy and Hold Returns : -0.08
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -8.72
Buy and Hold Returns : 1.62
Strategy Returns : -8.72
Buy and Hold Returns : 1.62
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -6.43
Buy and Hold Returns : 0.46
Strategy Returns : -6.43
Buy and Hold Returns : 0.46
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -44.43
Buy and Hold Returns : -0.67
Strategy Returns : -44.43
Buy and Hold Returns : -0.67
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.07
Buy and Hold Returns : 1

In [9]:
# Convert the results into dataframe sorted in a order which delivers greater returns
Results1 = pd.DataFrame({'Nifty_50_Stocks':nifty_stocks_name_intraday,'Returns':nifty_stocks_strategy_returns_intraday})
Results1.sort_values(by = 'Returns',ascending=False,inplace=True)

In [10]:
Results1

,Nifty_50_Stocks,Returns
5,KOTAKBANK.NS,0.07
24,BAJAJFINSV.NS,-0.20
13,HDFCLIFE.NS,-0.76
23,BAJFINANCE.NS,-1.73
12,INDUSINDBK.NS,-3.29
9,ULTRACEMCO.NS,-3.38
25,WIPRO.NS,-3.65
22,SHREECEM.NS,-3.68
10,CIPLA.NS,-4.15
28,TITAN.NS,-4.27


As per the above table,Closing on the same day does not work for all the stocks